In [0]:
import numpy as np
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F



In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
import torchvision.models as models



model = models.resnet18(False)
#model.fc = nn.Sequential(nn.Linear(model.fc.in_features,512),nn.ReLU(),nn.Linear(512,4))




In [0]:
num_ftrs = 512
model.avgpool = nn.AdaptiveAvgPool2d(1)
model.fc = nn.Linear(num_ftrs,4)

if train_on_gpu:
  model.cuda()
  
#model
  


In [0]:
from torch.utils.data.dataset import Dataset
from PIL import Image
import os



class TrainDataSet(Dataset):
     
    # Initialize your data, download, etc.
    def __init__(self,transform):
        self.train_df = pd.read_csv('g1drive/My Drive/Flipkart Challenge/training_set.csv')
        self.transform = transform
        self.len = self.train_df.shape[0]
    

    def __getitem__(self, index):
        img = cv2.imread(os.path.join('g1drive/My Drive/Flipkart Challenge/images/images',self.train_df.iloc[index,0]))
        ##img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        target_np = self.train_df.values[index,1:].astype(int)

        if self.transform is not None:
            img_as_tensor = self.transform(img)
        # Return image and the label
        return (img_as_tensor, torch.from_numpy(target_np))
      
    def __len__(self):
        return self.len


      


     



In [0]:
from torch.utils.data.dataset import Dataset
from PIL import Image
import os



class TestDataSet(Dataset):
     
    # Initialize your data, download, etc.
    def __init__(self,transform):
        self.train_df = pd.read_csv('g1drive/My Drive/Flipkart Challenge/test.csv')
        self.transform = transform
        self.len = self.train_df.shape[0]
    

    def __getitem__(self, index):
        img = cv2.imread(os.path.join('g1drive/My Drive/Flipkart Challenge/images/images',self.train_df.iloc[index,0]))
        ##img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        ##target_np = self.train_df.values[index,1:].astype(int)

        if self.transform is not None:
            img_as_tensor = self.transform(img)
        # Return image and the label
        return (img_as_tensor,self.train_df.iloc[index,0])
      
    def __len__(self):
        return self.len

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler


batch_size = 10
traindir = ''
valid_size = 0.2
num_workers = 0

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
preprocess = transforms.Compose([
   transforms.ToTensor(),
   normalize
])      
train_dataset = TrainDataSet(preprocess)

num_train = len(train_dataset)
indices = np.arange(0, num_train, 1)
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)

valid_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)


In [0]:
test_dataset = TestDataSet(preprocess)
batch_size = 10

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
     num_workers=num_workers)

In [0]:
import torch.optim as optim

# specify loss function
criterion = nn.MSELoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(),lr = 0.003)


In [0]:
from google.colab import drive
drive.mount('/content/g1drive',force_remount = True)

Mounted at /content/g1drive


In [0]:
!unzip 'g1drive/My Drive/images.zip' -d 'g1drive/My Drive/Flipkart Challenge/images/'

In [0]:
import os

dir = 'g1drive/My Drive/Flipkart Challenge/images/images'

list = os.listdir(dir) # dir is your directory path
number_files = len(list)
print(number_files)

56791


In [0]:
torch.cuda.empty_cache()

In [13]:
# number of epochs to train the model
model.load_state_dict(torch.load('g1drive/My Drive/Flipkart Challenge/model_flipkart(resnet50).pt'))
test_df = pd.read_csv('g1drive/My Drive/Flipkart Challenge/test.csv')
#print(test_df.head())
    ######################    
    # test the model #
    ######################
model.eval()
count = 0
for data,names in test_loader:
  # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data = data.cuda()
        
    output = model(data)
     
    if count%1000 == 0:
      print(count)
      
    for i in range(len(data)):
      name = names[i]
      count = count+1
      test_df.loc[test_df['image_name'] == name,'x1'],test_df.loc[test_df['image_name'] == name,'x2'],test_df.loc[test_df['image_name'] == name,'y1'],test_df.loc[test_df['image_name'] == name,'y2'] = output[i].cpu().detach().numpy()
    
    test_df.to_csv('answer-resnet-50.csv')
   

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [0]:
# number of epochs to train the model
model.load_state_dict(torch.load('g1drive/My Drive/Flipkart Challenge/Copy of model_flipkart1(1:AM).pt'))
n_epochs = 20 # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    count = 0
    for data,target in train_loader:
        # move tensors to GPU if CUDA is available
        #print(data.shape)
        #print(type(target[0][0]))
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target.float())
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        count = count + 10
        if count%1000 == 0:
          print(count)
          print(train_loss)
          print(target)
          print(output)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target.float())
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'g1drive/My Drive/Flipkart Challenge/model_flipkart(resnet18-after resize)')
        valid_loss_min = valid_loss

In [0]:
df = pd.read_csv('answer.csv')

In [0]:
print(df.iloc[0])

Unnamed: 0                            0
image_name    1467891670691DSC_0582.png
x1                              71.9158
x2                               568.94
y1                              63.7895
y2                              422.773
Name: 0, dtype: object
